In [6]:
from sys import getsizeof
from datetime import datetime
import sys, inspect, dis, asyncio
from functools import wraps
from unittest.mock import MagicMock
import random

# file touch
with open('hello_world.txt', 'a') as f:
    pass

def null_it(func):
    'Replace with no-op'
    def null(*args,**kwargs): pass
    return null

# A Taxonomy of Decorators: A-E

Andy Fundinger, Senior Engineer

EuroPython, Edinburgh, Scotland, July 29, 2018 

In [51]:
datetime.now()

datetime.datetime(2018, 6, 14, 15, 6, 23, 512798)

# Introduction 
* Who I am
* What is Bloomberg?
* What is this talk?
  - we use decorators, but when we go to talk about them it's hard to get them into categories 
  - with a common terminology we can discuss more easily
* Who is this talk for?  
  - Intermediate developers who can write decorators, but maybe aren't sure when and why
  - Architects who may need to work across teams to implement and manage their design

In [52]:
datetime.now() #40s

datetime.datetime(2018, 6, 14, 15, 7, 1, 534659)

## Decorator Syntax and Implementation 

Any decorator can be replaced with code in the decorated functions.  However, decorators allow this code to be reused and factored out of the functions.
- Decorators without the @ sign
- Function Decorators
- Class Decorators
- Decorators with arguments
- Decorators written as classes

Generally decorators insert a section of code of arbitrary complexity in a single line.  Debuggers generally skip over this code--for better or worse.

## Basic Syntax
### Decorators without the @ sign

The oldest decorators in Python are @staticmethod and @classmethod.  Dating back to Python 2.2 we used those like this:

In [53]:
class Util(object):
    def wibble():
        print("Wobble")
    wibble = staticmethod(wibble)

### The @ sign

The @ sign was added in 2.4 as a syntactic sugar for this pattern

In [54]:
class Util(object):
    @staticmethod
    def wibble():
        print("Wobble")
    @null_it
    def disabled():
        pass

We can write our own decorators as long as we accept a function and  return a replacement function.

In [1]:
def null_it(func):
    'Replace with no-op'
    def null(*args,**kwargs): pass
    return null

In [56]:
class Printer(object):
    @null_it
    def printOut(self, stuff):
        sys.stdout.write(stuff)
    def print_out(self, stuff):
        print(stuff)
              
Printer().print_out('Nonsense')        

Nonsense


In [57]:
Printer().printOut('something')

TypeError: 'NoneType' object is not callable

### Class Decorators

In Python 2.6 we can decorate classes too,

In [3]:
@null_it
class DeadClass:
    pass

DeadClass() is None

True

In [59]:
datetime.now() # 8 min

datetime.datetime(2018, 6, 14, 15, 9, 50, 367398)

### More advanced declarations

#### @wraps - impersonate the orginal function

In [68]:
from functools import wraps

def double_arg1(func):
    #@wraps(func)
    def wrapper(arg_one,*args, **kwargs):
        return func(arg_one*2, *args, **kwargs)
    wrapper.__wrapped__ = func
    wrapper.__module__ = func.__module__
    wrapper.__doc__ = func.__doc__
        
    return wrapper 

@double_arg1
def print_x(x):
    print(x)
    
print_x(1)
print_x('Hello')

AttributeError: 'function' object has no attribute 'co_code'

In [69]:
print_x

<function __main__.double_arg1.<locals>.wrapper>

### Decorators with arguments 

Decorators with arguments aren't actually doing anything all that special.  They simply call the function with the arguments and __that__ function returns the actual decorator.

In [70]:
def mult_arg(mult):
    def deco(func):
        def wrapper(arg_one, *args, **kwargs):
            return func(arg_one*mult, *args, **kwargs)
        return wrapper
    return deco

@mult_arg(3)
def print_x(x):
    print(x)
    
print_x(1)
print_x('Hello ')

3
Hello Hello Hello 


Notice, this is two closures, one for the argument (mult) and one for the function (func).  

### Decorators written as classes

Normally we write decorators that are closures, but there's no particular benefit to this.

In [71]:
import random

class trace_it:
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        print(args, kwargs)
        return self.func(*args, **kwargs)
        
@trace_it
def rand(min_val, max_val):
    return random.randint(min_val, max_val)
rand(10,30)

(10, 30) {}


27

In [72]:
datetime.now() # syntax total 12 min

datetime.datetime(2018, 6, 14, 15, 17, 15, 206398)

## A - Argument Changing Decorators 

* add or remove an argument when the function is called
* change the value or type of an argument at call time 
* similarly alter the return value

### Problems 

* calling the apparent signature does not actually work
* calling a function for a test requires injecting data to drive the decorator properly

### Example:  pytest.mark.parametrize()

In [74]:
import pytest

@pytest.mark.parametrize("test_input,expected", [
    ("3+5", 8),
    ("2+4", 6),
    ("6*9", 42),
])

def test_eval(test_input, expected):
    assert eval(test_input) == expected

### Example implementation -- adding the func name in the call

In [76]:
def fn_with_name(func):
    def wrapper(*args, **kwargs):
        func(func.__name__,*args,**kwargs)
    return wrapper
        
@fn_with_name
def self_aware(name, oth):
    print('{name}: {oth}'.format(name=name, oth=oth))
    
self_aware('here')
        

self_aware: here


In [77]:
datetime.now() # 3 min for  A

datetime.datetime(2018, 6, 14, 15, 20, 43, 521398)

## B - Binding Decorators

* implement the Descriptor Protocol to change how functions behave
* the standard library includes @staticmethod, @classmethod, and @property

### Problems

* creates an alternative to instance methods and attributes
* new language patterns arguably better fitting other languages 
* time shifts otherwise normal exceptions or introduces new ones

### Example: SQLAlchemy Hybrid Properties

In [78]:
#from sqlalchemy import func
hybrid_property= MagicMock()

class Interval(object):
    @hybrid_property
    def radius(self):
        return abs(self.length) / 2

    @radius.expression
    def radius(cls):
        return func.abs(cls.length) / 2

### Example implementation -- instance method

In [34]:
class instance_method:
    def __init__(self, func):
        self.func = func
    def __get__(self, inst, cls):
        if inst is None:
            raise TypeError(f'{self.func.__name__} is only valid on instances.')
        return self.func.__get__(inst, cls)
    
class GoodClass:
    @instance_method
    def simple_method(self):
        print('simple_method')
    def normal_method(self):
        print('normal_method')

In [35]:
GoodClass().normal_method()
GoodClass().simple_method()

normal_method
simple_method


In [36]:
GoodClass.normal_method()

TypeError: normal_method() missing 1 required positional argument: 'self'

In [37]:
GoodClass.simple_method()

TypeError: simple_method is only valid on instances.

In [32]:
GoodClass.normal_method

<function __main__.GoodClass.normal_method>

In [40]:
GoodClass.simple_method

TypeError: simple_method is only valid on instances.

In [85]:
datetime.now() # 6 min B

datetime.datetime(2018, 6, 14, 15, 26, 46, 120998)

## C - Control Flow Decorators

* change whether a function will be called and how many times

### Problems

* a predictable control flow now has a hidden conditional
* a single invocation might now lead to 0, 1, or many executions of the function

### Example: Retry Decorator

In [39]:
#from retrying import retry
retry = MagicMock()

@retry
def do_something_unreliable():
    if random.randint(0, 10) > 1:
        raise IOError("Broken sauce, everything is hosed!!!111one")
    else:
        return "Awesome sauce!"

print(do_something_unreliable())

<MagicMock name='mock()()' id='139994135494384'>


### Example implementation -- infinite retry

In [9]:
def infinite_retry(func):
    def wrapper(*args, **kwargs):
        while True:
            try:
                return func(*args, **kwargs)
            except RuntimeError as e:
                print(e)
    return wrapper

@infinite_retry
def random_fail(max_value):
    ret = random.randint(-100, max_value)
    if ret<0:
        raise RuntimeError("Invalid negative number {ret}".format(ret=ret))
    return ret

random_fail(10)

Invalid negative number -79
Invalid negative number -67


6

In [89]:
datetime.now() # 2 min C 

datetime.datetime(2018, 6, 14, 15, 28, 41, 753998)

## D - Descriptive Decorators

* add the decorated object to some sort of collection
* this collection will serve some other purpose such as:
  - documentation
  - dispatching
  - plugins

### Problems 

* it's unclear how dispatching will be done as a result of registration
* similarly it's hard to see where the registration is maintained

### Example pytest.marks

In [90]:
#import pytest
pytest = MagicMock()

@pytest.mark.webtest
def test_send_http():
    pass # perform some webtest test for your app
def test_something_quick():
    pass
def test_another():
    pass
class TestClass(object):
    def test_method(self):
        pass

### Example:  flask.app.route

In [92]:
#app = Flask(__name__)
app = MagicMock()

@app.route('/')
def hello_world():
    return 'Hello, World!'

### Example implementation -- qa list

In [97]:
import warnings 

to_qa=[]
def qa(func):
    to_qa.append(func.__name__)
    return func

@qa
def new_code(): pass

@qa
def refactored_code(): pass

def well_trusted_code(): pass

In [98]:
to_qa

['new_code', 'refactored_code']

In [99]:
new_code()

In [100]:
datetime.now() # 4 min D

datetime.datetime(2018, 6, 14, 15, 32, 10, 985998)

## E - Execution Decorators 

* reads the method/class code
* may reinterpret the source code to basically not be python

### Problems
* many
* and more

This truly means that the code you wrote is changed--by the decorator--to some other code that is then executed.  It might:
* be analyzed for dependencies
* have objects in the ast swapped out, injected or removed
* be recompiled with different rules 

### Example: cython

In [101]:
cython = MagicMock()

@cython.locals(a=cython.double, b=cython.double, n=cython.p_double)
def foo(a, b, x, y):
    n = a*b
    ...

### Example implementation -- code replacer

In [103]:
def replacer(old, new):
    def deco(func):
        source = inspect.getsource(func.__code__)
        lines = source.split('\n')
        new_source = lines[1]+'\n'+('\n'.join(lines[2:]).replace(old, new))
        exec(new_source,globals())
        return globals()[func.__name__]
    return deco

In [102]:
def sample(a, b):
    x = a + b
    y = x * 2
    print('Sample: ' + str(y))
sample(1,4)

Sample: 10


In [105]:
@replacer('b','b*3')
def sample(a, b):
    x = a + b
    y = x * 2
    print('Sample: ' + str(y))
sample(1,4)

Sample: 26


In [106]:
@replacer('a','a*3')
def sample(a, b):
    x = a + b
    y = x * 2
    print('Sample: ' + str(y))
sample(1,4)

Sa*3mple: 14


### Other tools available for execution decorators

* bytecode manipulation
* ast manipulation

In [107]:
datetime.now() # 7 min E 

datetime.datetime(2018, 6, 14, 15, 39, 17, 41998)

## Conclusion

Our Taxonomy
* Argument changing
  - @click.option
  - @flask.templated
  - @django.views.decorators.gzip.gzip_page
* Binding
  - @variants.primary
  - @pyramid.decorator.reify
* Control flow
  - @functools.lru_cache
  - @django.views.decorators.http.require_http_methods
  - @twisted.internet.defer.inlineCallbacks
* Descriptive
  - @numpy.testing.decorators.setastest
* Execution
  - @numba.jit
* ?

In [108]:
datetime.now()# 1 min conclusion

datetime.datetime(2018, 6, 14, 15, 39, 50, 994998)

## Feedback Please
* Pacing 
  - Syntax vs Examples
  - Examples themselves
* choice of examples
  - Real world examples
  - Example implementations
  - Execution decorator example

In [108]:
datetime.now()

datetime.datetime(2018, 6, 14, 15, 39, 50, 994998)

In [40]:
sample.__code__.co_code

b'|\x00\x00d\x01\x00\x14|\x01\x00\x17}\x02\x00|\x02\x00d\x02\x00\x14}\x03\x00t\x00\x00d\x03\x00t\x01\x00|\x03\x00\x83\x01\x00\x17\x83\x01\x00\x01d\x00\x00S'

In [41]:
dis.dis(sample.__code__.co_code)

          0 LOAD_FAST                0 (0)
          3 LOAD_CONST               1 (1)
          6 BINARY_MULTIPLY
          7 LOAD_FAST                1 (1)
         10 BINARY_ADD
         11 STORE_FAST               2 (2)
         14 LOAD_FAST                2 (2)
         17 LOAD_CONST               2 (2)
         20 BINARY_MULTIPLY
         21 STORE_FAST               3 (3)
         24 LOAD_GLOBAL              0 (0)
         27 LOAD_CONST               3 (3)
         30 LOAD_GLOBAL              1 (1)
         33 LOAD_FAST                3 (3)
         36 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
         39 BINARY_ADD
         40 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
         43 POP_TOP
         44 LOAD_CONST               0 (0)
         47 RETURN_VALUE


In [42]:
source = inspect.getsource(sample.__code__)
source

OSError: could not get source code

In [43]:
def sample(a, b):
    x = a + b
    y = x * 2
    print('Sample: ' + str(y))
sample(1,4)

Sample: 10


In [44]:
source = inspect.getsource(sample.__code__)
source

"def sample(a, b):\n    x = a + b\n    y = x * 2\n    print('Sample: ' + str(y))\n"

In [45]:
lines = source.split('\n')
new_source = lines[0]+'\n'+('\n'.join(lines[1:]).replace('b','b*3'))

In [46]:
new_source

"def sample(a, b):\n    x = a + b*3\n    y = x * 2\n    print('Sample: ' + str(y))\n"

In [47]:
glb = globals()
exec(new_source,glb)

In [48]:
glb['sample'](1,4), sample(1,4)

Sample: 26
Sample: 26


(None, None)

In [49]:
dis.dis(sample)

  2           0 LOAD_FAST                0 (a)
              3 LOAD_FAST                1 (b)
              6 LOAD_CONST               1 (3)
              9 BINARY_MULTIPLY
             10 BINARY_ADD
             11 STORE_FAST               2 (x)

  3          14 LOAD_FAST                2 (x)
             17 LOAD_CONST               2 (2)
             20 BINARY_MULTIPLY
             21 STORE_FAST               3 (y)

  4          24 LOAD_GLOBAL              0 (print)
             27 LOAD_CONST               3 ('Sample: ')
             30 LOAD_GLOBAL              1 (str)
             33 LOAD_FAST                3 (y)
             36 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             39 BINARY_ADD
             40 CALL_FUNCTION            1 (1 positional, 0 keyword pair)
             43 POP_TOP
             44 LOAD_CONST               0 (None)
             47 RETURN_VALUE
